In [17]:
%%writefile plug_ai.slurm
#!/bin/bash
#SBATCH --job-name=plug_ai
#SBATCH --output=plug_ai.out
#SBATCH --error=plug_ai.out
#SBATCH --gres=gpu:1
#SBATCH --ntasks-per-node=1
#SBATCH --nodes=1
#SBATCH --hint=nomultithread
#SBATCH --time=00:50:00
#SBATCH --qos=qos_gpu-dev
#SBATCH --cpus-per-task=10
#SBATCH --account=sos@v100
#SBATCH -C v100-16g


## load Pytorch module
module purge

module load pytorch-gpu/py3/1.10.1

## launch script on every node
set -x

# code execution
srun python ../plug_ai/__main__.py --config_file "../config_exemples/config_Brats.yaml"

Overwriting plug_ai.slurm


In [18]:
%%bash
# submit job
sbatch plug_ai.slurm

Submitted batch job 241300


In [19]:
from threading import Event
import signal
from IPython.display import clear_output

def quit(signo, _frame):
    print("Interrupted by %d, shutting down" % signo)
    exit.set()

for sig in ('TERM', 'HUP', 'INT'):
    signal.signal(getattr(signal, 'SIG'+sig), quit);

exit = Event()

sq = !squeue -u $USER -n plug_ai
tail = !tail plug_ai.out
while len(sq) >= 2 and not exit.is_set():
    
    clear_output(wait=True)
    print(sq[0], sq[1], sep='\n')
    print(*tail, sep='\n')
    
    sq = !squeue -u $USER -n plug_ai
    tail = !tail plug_ai.out

    exit.wait(1)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            241300   gpu_p13  plug_ai  ssos022  R       0:32      1 r10i7n3
[Epoch 1/1 |  Step_Epoch 2/8 | Loss 2.7129204273223877]
[Epoch 1/1 |  Step_Epoch 3/8 | Loss 2.6657700538635254]
[Epoch 1/1 |  Step_Epoch 4/8 | Loss 2.6930487155914307]
[Epoch 1/1 |  Step_Epoch 5/8 | Loss 2.716207265853882]
[Epoch 1/1 |  Step_Epoch 6/8 | Loss 2.6911604404449463]
[Epoch 1/1 |  Step_Epoch 7/8 | Loss 2.670424699783325]
[Epoch 1/1 |  Step_Epoch 8/8 | Loss 2.6973254680633545]
Epoch 0 finished
Execution over
>>> Complete execution in: 0:00:21.836335


In [13]:
!scancel -u $USER -n plug_ai